In [7]:
from collections import defaultdict
from preprocess import create_initial_data
import matplotlib.gridspec as gridspec
import warnings
from sklearn.manifold import LocallyLinearEmbedding
import os
import zipfile
import io
import pandas as pd
from dateutil import parser
import scipy.sparse
import numpy as np
import joblib
from tqdm import tqdm

# warnings.filterwarnings('ignore')
MOVIE_GENRE_AMOUNT = 12279
DIFF_GENRES=27
POPULIARY_TH_MOVIES=5000
import numpy as np
import seaborn as sns
import sklearn
import pickle
from IPython.display import display, HTML
import scipy.sparse
VOTING_THRESHOLD_USERS = 500
AMOUNT_MOVIES = 17770
AMOUNT_USERS=480189
sns.set()
FITLERED_USER_500 = 54238

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from pandas import read_pickle
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import seaborn as sns
import torchviz
import pickle
from sklearn.preprocessing import scale
import hiddenlayer as hl
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
import matplotlib as mpl
import plotly.express as px

plt.rcParams['figure.figsize'] = [23, 13]
plt.rcParams['figure.dpi'] = 100 # from mpl_toolkits.mplot3d import Axes3D
mpl.rcParams['xtick.major.pad'] = 8
mpl.rcParams['ytick.major.pad'] = 8

from matplotlib import ticker
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-1,1))

def save_pickle(name,obj):
    with open(name+'.pickle', 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def get_genres_of_movies():
	"""
	This function reads the file named 'netflix_genres.csv' which has a mapping between movie id and its genre.
	It prints all unique genres and return a dataframe which is a one-hot encoding - the dataframe contains all
	genres as columns, and 1 if this movie is from this genre, and 0 otherwise (because a movie usually corresponds
	to more than one genre)
	:return:
	"""
	df_of_genres = pd.read_csv('netflix_genres.csv')
	all_genres = set()
	for movie_genre in df_of_genres['genres'].to_list():
		all_genres.update(movie_genre.split('|'))
	print("all genres are:")
	print(all_genres)
	print("Number of genres is: ")
	print(len(all_genres))

	df_of_movies_and_all_genres = pd.DataFrame(columns=all_genres)
	for idx, row in df_of_genres.iterrows():
		movie_id = row[0]
		movie_genres = row[1].split('|')
		for movie_genre in movie_genres:
			df_of_movies_and_all_genres.loc[movie_id, movie_genre] = 1
	df_of_movies_and_all_genres.fillna(0, inplace=True)
	return df_of_movies_and_all_genres


def np_display(X):
    display(pd.DataFrame(X))

In [63]:
mat_of_movies_and_users,movie_genres_mat= genres=create_initial_data(True,True)

Started loading data from disk
Finished loading data from disk


In [ ]:
df_of_movies_info = pd.read_csv('movie_titles.csv', error_bad_lines=False, encoding='latin-1', index_col=0,
                                names=['year_of_release', 'title'])

df_of_movies_genres = get_genres_of_movies()
df_of_movies_info = df_of_movies_info.join(df_of_movies_genres)
df_of_movies_info.fillna(0, inplace=True)
print("started")
transope_mat_user = scipy.sparse.load_npz('transope_mat_user_comp' + '.npz').tolil()
print("ended")


In [ ]:
#Amount of times each user rated a movie.
votes_users=np.zeros(AMOUNT_USERS)
for i in range(0,AMOUNT_USERS):
    votes_users[i]=transope_mat_user.getrow(i).count_nonzero()
save_pickle('votes_users',votes_users)


In [3]:
#Dropping users below threshold
votes_users=pd.read_pickle("votes_users.pickle")
votes_users_pd=pd.DataFrame(votes_users)
print("reached!")
votes_users_th=votes_users_pd[votes_users_pd>VOTING_THRESHOLD_USERS]
(votes_users_th.dropna(inplace=True))



reached!


In [ ]:
#Filtering only the relevant users from the original sparse matrix.
above_th_users=(votes_users_th.index)
np_display(above_th_users)
display(votes_users_th)
user_move_pst_th=scipy.sparse.lil_matrix((FITLERED_USER_500,AMOUNT_MOVIES))
for i,j in zip(above_th_users,np.arange(FITLERED_USER_500)):
    user_move_pst_th[j,:]=transope_mat_user.getrow(i)
    if j%1000==0:
        print(j)
        
scipy.sparse.save_npz('post_threshhold_users',user_move_pst_th.tocsr(),compressed=True)

In [4]:
users_filtered=scipy.sparse.load_npz('post_threshhold_users.npz')


In [5]:
genres=pd.read_pickle('genres_orthgonal.pickle')
df_of_movies_info = pd.read_csv('movie_titles.csv', error_bad_lines=False, encoding='latin-1', index_col=0,
															names=['year_of_release', 'title'])
movies = df_of_movies_info.join(genres)
movies.fillna(0, inplace=True)

In [ ]:
#Create the feature vector for each user.
current_data=scipy.sparse.lil_matrix((FITLERED_USER_500,58))
c=0
for i in range(FITLERED_USER_500):
    c+=1
    curr_row=users_filtered.getrow(i)
    indexs=curr_row.nonzero()[1]
    ratings=curr_row.data
    user_ortgh_vector=np.zeros(27)
    per_genre=np.zeros(27)
    for movie_id,rating in zip(indexs,ratings):
        try:
            curr_orthg=genres.loc[movie_id]
        except:
            continue
        user_ortgh_vector+=curr_orthg*rating
        per_genre+=curr_orthg
        
    dist_user_ratings=user_ortgh_vector/per_genre-2.5
    user_popuarity_dist=(user_ortgh_vector/user_ortgh_vector.sum())*100 # Scaling the feature.
    user_profile=np.zeros(58)
    user_profile[0]=dist_user_ratings.mean()
    user_profile[1]=dist_user_ratings.std()
    user_profile[2]=user_popuarity_dist.mean()
    user_profile[3]=user_popuarity_dist.std()
    user_profile[4:]=np.hstack([user_popuarity_dist,dist_user_ratings])
    
    user_profile=np.nan_to_num(user_profile)
    current_data[i,:]=user_profile
    if c%100==0:
        print(c)
        
save_pickle('user_feature_v',current_data)

In [2]:
#Transfer the data using the denoisinsg autoencdoer.
import torch
import denoising_ae
users, test_loader, train_loader = denoising_ae.data_prep()
net = denoising_ae.Denoising_AE()
#
print(net.enc15)

Linear(in_features=15, out_features=10, bias=True)


In [ ]:
d=torch.load("weights_ae/weights_curr30")
a=net.load_state_dict(d)
dd=np.zeros((FITLERED_USER_500,10))

dataloader_users=DataLoader(np.array(users).astype(np.float32))
net.eval()
for data,i in zip(dataloader_users,range(FITLERED_USER_500)):
    dd[i]=((net.encoder(data)).detach().numpy())
    
#     a=(dd[i])
#     b=(net.encoder(torch.Tensor(np.array(users.iloc[i]).astype(np.float32))).detach().numpy())
#     print(a==b)
    
#     if i==1000 :
#         break
        

# all_users_dim3=pd.DataFrame(dd[:,:3],columns=['x','y','z'])
# save_pickle("final_dataset_dim3",all_users_dim3)





,x,y,z
count,5.423800e+04,5.423800e+04,5.423800e+04
mean,5.833790e+31,5.773754e+31,5.352835e+31
std,7.364536e+30,7.288747e+30,6.757382e+30
min,1.136542e+30,1.124846e+30,1.042842e+30
25%,5.537920e+31,5.480929e+31,5.081358e+31
50%,5.971018e+31,5.909570e+31,5.478750e+31
75%,6.294323e+31,6.229547e+31,5.775400e+31
max,8.225668e+31,8.141017e+31,7.547520e+31
